In [14]:
import numpy as np
from pathlib import Path
from sklearn.compose import TransformedTargetRegressor
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [15]:
DATA = {
    "lung":    dict(X="lung_features.npy",    y="lung_target.npy"),
    "bladder": dict(X="bladder_features.npy", y="bladder_target.npy")
}

In [ ]:
def flatten(x):  # (n, t, p)  →  (n, t·p)
    n, t, p = x.shape
    return x.reshape(n, t * p)

In [17]:
def mlp_baseline(name, seed=42, k=1024):
    X = flatten(np.load(DATA[name]["X"]))
    y = np.load(DATA[name]["y"])

    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=seed)

    pipe = Pipeline([
        ("const", VarianceThreshold()),
        ("kbest", SelectKBest(mutual_info_regression,
                              k=min(k, X.shape[1]))),
        ("xscale", StandardScaler(with_mean=False)),
        ("mlp",   TransformedTargetRegressor(
            regressor = MLPRegressor(hidden_layer_sizes=(128, 64, 32),
                                      activation="relu",
                                      alpha=1e-3,
                                      learning_rate_init=1e-3,
                                      max_iter=400,
                                      early_stopping=True,
                                      n_iter_no_change=10,
                                      random_state=seed),
            transformer = StandardScaler()))
    ])

    pipe.fit(Xtr, ytr)
    pred = pipe.predict(Xte)

    return dict(
        MAE = mean_absolute_error(yte, pred),
        MSE = mean_squared_error(yte, pred),
        R2  = r2_score(yte, pred)
    )

In [18]:
for ds in ("lung", "bladder"):
    scores = mlp_baseline(ds)
    print(f"{ds.upper():8}  "
          f"MAE={scores['MAE']:.6f}  "
          f"MSE={scores['MSE']:.8f}  "
          f"R²={scores['R2']:+.3f}")

LUNG      MAE=0.000887  MSE=0.00000215  R²=-0.333
BLADDER   MAE=0.000274  MSE=0.00000019  R²=+0.226
